# by *convention* __test__ing should be interactive.

In [1]:
    ...; o = __name__ == '__main__'; ...;

Interactive testing is manual quality assurance of notebook source.  This extension introduces an automated testing approach that combines __doctest__, __unittest__, and __hypothesis__ to test interactive strings, functions, and classes.

The conventions set forth by `rites.hypothesis` will encourage better types annotations, docstrings, and reusabulity.

## Testing During Interactive Programming Is Efficient

In interactive mode, the `testing` extension will execute tests when:
    
* An unassigned string literatal containing doctests.
* Docstrings in functions definitions and class defitions.
* Functions containing complete annotations.
* Classes containing runTest
* TestCase classes

In [2]:
    from doctest import DocTestCase, DocTestSuite, DocTest, DocTestParser, DocTestFinder, testmod
    from ast import *
    from functools import wraps, partial
    from dataclasses import dataclass, field
    from IPython import get_ipython
    from inspect import *
    from hypothesis import given, strategies, strategies as st, assume, HealthCheck, Verbosity, settings, find
    from unittest import TestCase, TextTestRunner, TestSuite
    __all__ = 'infer',
    
    def infer(object, ghetto=True, module=None)->('test', dict, any):
        """Use the hypothesis inference systems to create automated types from the annotations or ghetto typing.
        
        
        >>> def f(int, b:int): ...
        >>> test, annotations, returns = infer(f)
        >>> assert all(str in annotations for str in ('int', 'b'))
        """
        from unittest import FunctionTestCase
        from inspect import getfullargspec
        
        spec = getfullargspec(object)
        
        annotations = dict(**spec.annotations)
        returns = annotations.pop('return', None)
        
        if not spec.args: 
            return FunctionTestCase(object), annotations, returns
        
        module = module or __import__('__main__')
        
        if ghetto:
            for arg in spec.args:
                if arg in annotations: continue
                try:
                    thing = eval(arg, vars(module))
                except NameError: continue
                if isinstance(thing, (type, list)):
                    annotations[arg] = thing
                
        if not (spec.defaults or spec.kwonlydefaults):
            try:
                annotations = {
                    str: st.from_type(object) if isinstance(object, type) and getattr(object, '__name__', '') != 'object'
                    else object if isinstance(object, st.SearchStrategy)
                    else st.one_of(list(map(st.just, object)))
                    for str, object in annotations.items()
                }
                if annotations:
                    return FunctionTestCase(given(**annotations)(object)), annotations.copy(), returns
            except: ...
                
        return None, annotations, returns
        


In [3]:
    def test(Testing, *, context=None, module=None):
        """Test an object contains 
        
        >>> testing = Testing()
        >>> testing.objects = ['f']
        >>> def f():...
        >>> demo = __import__('types').ModuleType('demo')
        >>> test(testing, module=__import__('__main__'), context=setattr(demo, 'f', f) or demo)
        """
        from types import ModuleType
        global mod
        
        module, context = ModuleType('__main__'), context or __import__('__main__')
        tests = list()
        mod = module
        module.__test__ = getattr(module, '__test__', {})
        
        while Testing.objects:
            name = Testing.objects.pop(0)
            object = getattr(context, name)
            setattr(module, name, object)
            if isinstance(object, type):
                if issubclass(object, TestCase): 
                    tests.append(object())
                elif hasattr(object, 'runTest'):
                    def runTest(): return object.runTest(object)
                    tests.append(FunctionTestCase(wraps(object.runTest)(runTest)))
            elif callable(object):
                test, annotations, returns = infer(object)
                if (returns is not None) and callable(returns) ^ isinstance(returns, type)and len(annotations) is 1:
                    try:
                        find(*annotations.values(), lambda x: returns(object(x)))
                    except:
                        assert False, f"""NoSuchExample for {object} satifies {returns} """
                elif test: tests.append(test)
                    
            if getattr(object, '__name__', None):
                tests.extend(map(DocTestCase, DocTestFinder().find(object)))
                
        try:
            mod = module
            runner = TextTestRunner(verbosity=1)
            tests and runner.run(TestSuite(set(tests)))            
        except: ...
        Testing.objects = []

## The NodeTransformer

... accumulates the testable functions and classes in the parsed ast.  It must be `callable` to use with `get_ipython().events.callback['post_run_cell']`

In [4]:
    from dataclasses import dataclass, field

    @dataclass
    class Testing(NodeTransformer):
        """Testing must be callable so it can be using as an {IPython.core.events.EventManager}
        >>> assert callable(Testing())
        """
        objects = list()
        
        def visit_FunctionDef(Testing, node): 
            """Identify FunctionDef and ClassDef as potentially testable objects.
            
            >>> visitor = Testing()
            >>> assert visitor.visit(ast.parse('def f(): ...'))
            >>> assert visitor.objects
            """
            Testing.objects.append(node.name)
            return node
        
        visit_ClassDef = visit_FunctionDef
        
        __call__ = test

## Extensions

    %unload_ext rites.hypothesis

In [5]:
    def unload_ipython_extension(ip=None):
        """>>> unload_ipython_extension()"""
        transformers = []
        ip = ip or get_ipython()
        ip.events.callbacks['post_run_cell'] = [object for object in ip.events.callbacks['post_run_cell'] if not isinstance(object, Testing)]        

    %reload_ext rites.hypothesis

In [6]:
    def load_ipython_extension(ip=get_ipython()):
        """
        >>> ip = load_ipython_extension()
        >>> assert any(isinstance(object, Testing) for object in ip.ast_transformers)
        """
        settings.register_profile('ip', settings(
            suppress_health_check=(HealthCheck.return_value,),
            verbosity=Verbosity.normal,))
        unload_ipython_extension(ip)
        object = Testing()
        ip.ast_transformers.append(object)        
        ip.events.register('post_run_cell', object)
        settings.load_profile('ip')
        return ip

## Examples

In [7]:
    if o: load_ipython_extension()

In [8]:
    def a_function_without_tests(a, b):
        return a+b

In [9]:
    def any_function_with_a_True_docstring_is_doctested(a, b):
        """ """

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [10]:
    def a_function_with_a_doctest(a, b):
        """Test {a_function_with_a_doctest} with {doctest}
        
        >>> assert a_function_with_a_doctest(10, 32) is 42
        """
        return a+b

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [11]:
    from hypothesis.errors import UnsatisfiedAssumption
    ct = 0
    def test_a_function_with_edge_cases(a: int, b: int):
        """{test_a_function_with_edge_cases} uses the type annotations to infer a hypothesis strategy to test the function
        
        >>> assert test_a_function_with_edge_cases(10, 32) is 42
        >>> try:
        ...     test_a_function_with_edge_cases(0, 0)
        ...     assert None, 'This should not throw an Exception'
        ... except UnsatisfiedAssumption:
        ...     assert True, 'Hypothesis throws a special unsatisfiable error.'
        """
        global ct
        ct += 1
        __import__('hypothesis').assume(a or b)
        assert a+b
        return a+b

..
----------------------------------------------------------------------
Ran 2 tests in 0.182s

OK


In [12]:
    f"{test_a_function_with_edge_cases} was automatically evaluated {ct} times."

'<function test_a_function_with_edge_cases at 0x10cc35d08> was automatically evaluated 103 times.'

## Using `doctest`

In [33]:
    o and __import__('doctest').testmod(verbose=2), """for all of the doctests."""

.

Trying:
    assert callable(Testing())
Expecting nothing
ok
Trying:
    visitor = Testing()
Expecting nothing
ok
Trying:
    assert visitor.visit(ast.parse('def f(): ...'))
Expecting nothing
ok
Trying:
    assert visitor.objects
Expecting nothing
ok
Trying:
    assert a_function_with_a_doctest(10, 32) is 42
Expecting nothing
ok
Trying:
    def f(int, b:int): ...
Expecting nothing
ok
Trying:
    test, annotations, returns = infer(f)
Expecting nothing
ok
Trying:
    assert all(str in annotations for str in ('int', 'b'))
Expecting nothing
ok
Trying:
    ip = load_ipython_extension()
Expecting nothing
ok
Trying:
    assert any(isinstance(object, Testing) for object in ip.ast_transformers)
Expecting nothing
ok
Trying:
    testing = Testing()
Expecting nothing
ok
Trying:
    testing.objects = ['f']
Expecting nothing
ok
Trying:
    def f():...
Expecting nothing
ok
Trying:
    demo = __import__('types').ModuleType('demo')
Expecting nothing
ok
Trying:
    test(testing, module=__import__('__main


----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


(TestResults(failed=0, attempted=18), 'for all of the doctests.')

## nbconvert execution

    __import__('nbconvert').preprocessors.execute.ExecutePreprocessor().preprocess(__import__('nbformat').read('hypothesis.ipynb', 4),{})